<font size = 6, color = gold> GOLD

# Sorting out basic design issues

## Start Focus on Bidder Side - Make bidding decisions

In [48]:
import numpy as np
import random

class User:
    def __init__(self):
        self.probability2 = random.uniform(0, 1)

    def show_ad(self):
        return  np.random.choice((True, False), p = [self.probability2, 1-self.probability2])

class Auction:
    
    auction_history = [] # for the auction exchange only
    balances = {}
    
    def __init__(self, users, bidders):
        self.users = users
        self.bidders = bidders
        
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        print("*"*100, '\n')
        print('Beginning balances : ',self.balances, '\n','-'*100)
        
        
    def execute_round(self):
        
        ##### if the -ve balance of ALL bidders is < -1000, then break
        
        all_invalid_bidders = any(x > -1000 for x in self.balances.values())
        
        while not all_invalid_bidders:
            print("no bidders are qualifed as all balances are below -1000")
            break
        
        else:
            # print('starting balances2: ', self.balances2, '\n','-'*100)
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            # 1. SELECT USER (from the pool of users) ----------------------------------

            chosen_user = random.randint(0, len(self.users)-1)
            print('Selected User: ', id(chosen_user), '\n','-'*100) 

            # 2. COLLECT BIDS (send to all bidders) --------------------------------------------
            for bidder in self.bidders:
                #if self.balances[bidder] > -1000:
                #balances = {bidder: 0 for bidder in bidders} ########### NEW LINE
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            print("Current bidding Round: ", bidding_round, '\n','-'*100)

            # 3. DETERMINE WINNING BID --------------------------------------------
            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))


    #        print('bids list', bids_list, '\n','-'*100)

            ###### SORT THE Bids List based on price ##########################################
            sorted_list = sorted(bids_list, key=lambda t:t[1])
            print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis - creating a copy with id(bidder) --------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
    #        print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ####### Select second highest price ###############################################
            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ######## Determine if there are more than 1 bidders with same price ######################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            print('Winning Bidder (FINAL): ', id(winning_bidder), '\n','-'*100)
            print('Winning Price (FINAL): ', winning_price, '\n','-'*100)

            ############################################################################################
            # 4. Validate USER CLICK after SHOW AD  --------------------------------------------
            ad_result = self.users[chosen_user].show_ad()
            print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            # 5.NOTIFY BIDDER & UPDATE BALANCES --------------------------------------------

            print("Bidding Outcomes", '\n','-'*100)
            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1
                    # balances[bidder] -= winning_price
                    # balances[bidder] += 1                

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price
                    # balances[bidder] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)
                # print('balances at end of bidding round', balances,'\n','-'*100)
        
class Bidder:
    
    #### maintain account balance here as well?

    # BIDDER CREATION -------------- initiated by GAME runner / Developer ------------------------
    
    def __init__(self, num_users, num_rounds):

        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0

        self.bid_y_n = True
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins = {i: 0 for i in range(num_users)} # if bidder wins
        #print('bid wins', self.bid_wins)
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}
        
        #Transaction records
        self.bid_participate_history = [] # maintains bidderid, userid, bidding round, bid price
        self.bid_win_history = [] # won bid
        self.bid_win_user_click_history = [] # won bid and user clicked
        self.bid_lost_history = [] # lost bid, but know the user and price
        
        ############
        #? Winning bid price / user (because you get notified of winning bids) -- 
        ###### you know the user and winning bid (just not the click result)
        
    # SUBMIT BID  ------------ initiated by auction ------------------------------------
    
    def bid(self, user_id):
        self.bidding_round += 1
        #self.bid_participate = 0
        self.user_id = user_id
        default_bid_price = random.randint(1000, 10000)/2000 #random.uniform(0, 1)
        # bid_price = self.default_bid_price + (self.bidding_round)*0.1
        bid_price = default_bid_price
        #bid_price = 0.5
        self.bid_participate[self.user_id] = (self.bid_participate).get(self.user_id, 0)+1
        self.bid_participate_history.append([id(self), self.user_id, self.bidding_round, bid_price])
        return bid_price
    
    def bid_strategy(self):
        pass
    
    # NOTIFIED OF BID OUTCOME ------------ initiated by auction ------------------------------------
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        if auction_winner:
            #self.bid_participate_history.append([self.user_id, bid_y_n_won+1])
            self.bid_wins[self.user_id] = (self.bid_wins).get(self.user_id, 0)+1
            # self.bid_win_history.append([self.user_id, bid_y_n_won+1])
            self.bid_win_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            if clicked:
                self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
                #self.bid_win_user_click_history.append([self.user_id, self.bidding_round])
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1
            self.bid_lost_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

            # NOTIFICATION: bidder.notify(False, winning_price, None)
            pass

In [49]:
b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3])
#auction = Auction( [User(), User()], [b1, b2, b3])

auction.execute_round()
# auction.execute_round()
# auction.execute_round()
# auction.execute_round()

**************************************************************************************************** 

Beginning balances :  {140410013059632: 0, 140410013059680: 0, 140408924663520: 0} 
 ----------------------------------------------------------------------------------------------------
Selected User:  140409997158832 
 ----------------------------------------------------------------------------------------------------
Current bidding Round:  1 
 ----------------------------------------------------------------------------------------------------
Sorted bids list [(140410013059632, 1.351), (140410013059680, 3.6555), (140408924663520, 3.693)] 
 ----------------------------------------------------------------------------------------------------
Winning Bidder (FINAL):  140408924663520 
 ----------------------------------------------------------------------------------------------------
Winning Price (FINAL):  3.6555 
 ----------------------------------------------------------------------

In [60]:
print('Bidding Rounds', b1.bidding_round)
#print('Bid Participate', b1.bid_participate)
print('Bid Participate Sorted', {k: v for k, v in sorted(b1.bid_participate.items(), key=lambda item: item[0])})
#print('Bid Wins', b1.bid_wins)
print('Bidding Wins Sorted', {k: v for k, v in sorted(b1.bid_wins.items(), key=lambda item: item[0])})
#print('User Clicks', b1.bid_win_user_clicks)
print('User Clicks Sorted', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
#print('Bids Lost', b1.bid_lost)
print('Bids Lost Sorted', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])}) 

Bidding Rounds 101
Bid Participate Sorted {0: 15, 1: 10, 2: 6, 3: 9, 4: 10, 5: 8, 6: 13, 7: 10, 8: 10, 9: 10}
Bidding Wins Sorted {0: 5, 1: 4, 2: 3, 3: 4, 4: 3, 5: 1, 6: 1, 7: 3, 8: 2, 9: 1}
User Clicks Sorted {0: 0, 2: 2, 4: 2, 5: 1, 6: 1, 7: 2, 8: 1, 9: 1}
Bids Lost Sorted {0: 10, 1: 6, 2: 3, 3: 5, 4: 7, 5: 7, 6: 12, 7: 7, 8: 8, 9: 9}


In [65]:
print('occurances', len(b1.bid_win_history))
b1.bid_win_history # 1.bidding_round, 2. User_id, 3. auction_winner 4.self.price  5. self.clicked

occurances 27


[[7, 0, True, 3.8105, False],
 [9, 7, True, 3.2755, False],
 [11, 4, True, 1.319, False],
 [17, 7, True, 1.236, True],
 [18, 2, True, 3.757, True],
 [23, 4, True, 1.913, True],
 [25, 6, True, 1.3065, True],
 [26, 0, True, 4.2855, False],
 [27, 3, True, 1.988, False],
 [30, 1, True, 0.9955, False],
 [33, 8, True, 1.2555, True],
 [34, 1, True, 3.277, False],
 [39, 7, True, 3.851, True],
 [41, 9, True, 3.797, True],
 [43, 4, True, 2.6375, True],
 [47, 1, True, 1.2645, False],
 [48, 0, True, 4.4785, False],
 [51, 2, True, 3.736, False],
 [55, 0, True, 4.201, False],
 [66, 2, True, 3.0415, True],
 [74, 5, True, 1.4115, True],
 [86, 8, True, 1.711, False],
 [92, 1, True, 1.065, False],
 [93, 3, True, 1.016, False],
 [96, 3, True, 3.718, False],
 [98, 3, True, 3.1725, False],
 [99, 0, True, 3.766, False]]

In [66]:
print('occurances',len(b1.bid_lost_history))
b1.bid_lost_history # 1.bidding_round, 2. User_id, 3. auction_winner 4.self.price  5. self.clicked

occurances 74


[[1, 5, False, 3.6555, None],
 [2, 7, False, 3.0985, None],
 [3, 6, False, 1.9525, None],
 [4, 6, False, 4.7835, None],
 [5, 8, False, 2.55, None],
 [6, 2, False, 3.337, None],
 [8, 9, False, 1.4465, None],
 [10, 3, False, 4.252, None],
 [12, 6, False, 1.323, None],
 [13, 6, False, 4.504, None],
 [14, 0, False, 2.1735, None],
 [15, 0, False, 1.857, None],
 [16, 0, False, 2.427, None],
 [19, 0, False, 1.718, None],
 [20, 6, False, 2.519, None],
 [21, 9, False, 2.1505, None],
 [22, 5, False, 4.1305, None],
 [24, 3, False, 2.0185, None],
 [28, 2, False, 4.367, None],
 [29, 9, False, 2.8725, None],
 [31, 7, False, 2.9985, None],
 [32, 6, False, 2.836, None],
 [35, 7, False, 3.8315, None],
 [36, 9, False, 2.879, None],
 [37, 0, False, 2.5695, None],
 [38, 3, False, 2.876, None],
 [40, 5, False, 4.11, None],
 [42, 3, False, 1.6245, None],
 [44, 5, False, 2.6955, None],
 [45, 1, False, 2.562, None],
 [46, 7, False, 1.22, None],
 [49, 9, False, 1.1295, None],
 [50, 0, False, 4.2475, None],
 [5

In [50]:
[auction.execute_round() for i in range(100)]

Selected User:  140409997158896 
 ----------------------------------------------------------------------------------------------------
Current bidding Round:  2 
 ----------------------------------------------------------------------------------------------------
Sorted bids list [(140410013059632, 2.6815), (140410013059680, 3.0985), (140408924663520, 4.571)] 
 ----------------------------------------------------------------------------------------------------
Winning Bidder (FINAL):  140408924663520 
 ----------------------------------------------------------------------------------------------------
Winning Price (FINAL):  3.0985 
 ----------------------------------------------------------------------------------------------------
User Clicked on Ad?:  True 
 ----------------------------------------------------------------------------------------------------
Bidding Outcomes 
 ----------------------------------------------------------------------------------------------------
1404100

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [53]:
auction.balances

{140276743991200: -1000.3605,
 140276743990432: -1000.5714999999998,
 140276743982816: -1000.3000000000001}

In [7]:
b1.bidding_round

101

In [14]:
b1.bid_participate_history #bidderid, userid, bidding round, bid price

[[140467995445472, 5, 1, 2.633]]

In [ ]:
b2.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
b3.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
[auction.execute_round() for i in range(10)]

In [ ]:
b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()], [b1, b2, b3])
auction.execute_round()

In [ ]:
print('b1 bid is: ', b1.bid(0x7fb6581c8ca0))
print('b2 bid is: ', b2.bid(0x7fb6581c8ca0))
print('b3 bid is: ', b3.bid(0x7fb6581c8ca0))

In [ ]:
u1 = User()
id(u1)

In [ ]:
user_wins = {i: 0 for i in range(10)}
user_wins

In [17]:
bids_list = [(1,2), (7,8), (5,6),(3,4) ]

In [18]:
sorted_list = sorted(bids_list, key=lambda t:t[1])
sorted_list

[(1, 2), (3, 4), (5, 6), (7, 8)]

In [19]:
winning_price = (sorted_list)[-2][1]
winning_price

6

In [21]:
winning_bid = (sorted_list)[-1][0]
winning_bid

7

In [22]:
list2 = [(1,2)]
sorted_list = sorted(list2, key=lambda t:t[1])
sorted_list

[(1, 2)]

In [23]:
winning_price = (sorted_list)[0][-1]

In [24]:
winning_price

2

In [26]:
winning_bid = (sorted_list)[0][0]
winning_bid

1